In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import missingno
#from sklearn.model_selection import train_test_split
#from sklearn import decomposition
#from sklearn import datasets
#from sklearn.cluster import KMeans
#from sklearn.preprocessing import MinMaxScaler
#import statsmodels.api as sm

HAHA, das ist ein Test